In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [59]:
import mlflow
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [36]:
data = pd.read_parquet('d:/demand-forecast-SQGroup/data/saleBYA_bestFeatures.parquet')
df = data.copy()

In [37]:
df.head()

,date_month_start,net_price_window_7_std,net_price_lag_28,net_price_lag_7,net_price_lag_3,qtym_expanding_std,date_day_of_month,date_month,net_price,date
0,0,5.136896e+06,26272324.0,1.914946e+05,2599070.50,181123.941914,8,8,5042942.0,2019-08-08
1,0,5.042066e+06,15105817.0,1.105840e+07,4009107.25,179949.733415,10,8,2932270.5,2019-08-10
2,0,4.480915e+06,5594980.5,5.000808e+06,4451132.00,176910.642883,12,8,1809937.5,2019-08-12
3,0,4.958484e+06,9242950.0,2.071218e+04,5042942.00,182988.967644,13,8,3067519.0,2019-08-13
4,0,5.091923e+06,9014580.0,2.599070e+06,2932270.50,182449.881234,14,8,5123492.0,2019-08-14


In [38]:
folds = np.load('d:/demand-forecast-SQGroup/data/folds.npz')

In [39]:
train = df.loc[folds['train_2']]
test = df.loc[folds['test_2']]

In [40]:
train.shape

(841, 10)

In [41]:
test.shape

(280, 10)

In [42]:
X = train.drop(columns=['date', 'net_price'])
y = train['net_price']

In [32]:
Xt = test.drop(columns=['date', 'net_price'])
yt = test['net_price']

In [45]:
X

,date_month_start,net_price_window_7_std,net_price_lag_28,net_price_lag_7,net_price_lag_3,qtym_expanding_std,date_day_of_month,date_month
0,0,5.136896e+06,2.627232e+07,1.914946e+05,2599070.50,181123.941914,8,8
1,0,5.042066e+06,1.510582e+07,1.105840e+07,4009107.25,179949.733415,10,8
2,0,4.480915e+06,5.594980e+06,5.000808e+06,4451132.00,176910.642883,12,8
3,0,4.958484e+06,9.242950e+06,2.071218e+04,5042942.00,182988.967644,13,8
4,0,5.091923e+06,9.014580e+06,2.599070e+06,2932270.50,182449.881234,14,8
...,...,...,...,...,...,...,...,...
836,0,2.035232e+06,4.122544e+06,3.239888e+04,4586322.50,127288.451741,3,7
837,0,2.093357e+06,1.410594e+07,2.700934e+06,5069513.50,127236.588362,4,7
838,0,2.062016e+06,4.366118e+06,3.965883e+06,16610658.00,127163.299990,5,7
839,0,1.852521e+06,4.973830e+06,5.084555e+06,5036285.50,127090.038666,6,7


<center><b> Experiments </b></center>

In [4]:
# mlflow.create_experiment('Net Price Forecasting')

'926698075080674526'

In [62]:
mlflow.set_experiment('Net Price Forecasting')
with mlflow.start_run():
    model = KNeighborsRegressor(n_neighbors=9, weights='distance', metric='euclidean')
    model.fit(X, y)


    ### log params
    mlflow.log_params(dict(n_neighbors=9, weights='distance', metric='euclidean'))
    
    ### log metrics
    mlflow.log_metrics(
        {
            'r2': r2_score(model.predict(Xt), yt),
            'mse': mean_absolute_percentage_error(model.predict(Xt), yt)
        }
    )
    
    ### log model